In [23]:
# Please use this to connect your GitHub repository to your Google Colab notebook
# Connects to any needed files from GitHub and Google Drive

# Remove Colab default sample_data
# !rm -r ./sample_data


In [24]:
# Data
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from google.cloud import storage

# Models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torchvision.models import resnet50
import torch.optim as optim
import torch.nn as nn
import torch

# Absolute, stable local directory (fixes relative-path surprises)
RAW_DIR = Path("./data/raw").resolve()
RAW_DIR.mkdir(parents=True, exist_ok=True)
print("RAW_DIR:", RAW_DIR)



RAW_DIR: /content/data/raw


In Google Cloud Storage, a blob is just one stored object in a bucket — basically a file-like thing (an image, CSV, zip, etc.) plus its metadata.

So in your code:
	•	bucket.list_blobs(prefix=gcs_folder_prefix) returns an iterator of blob objects
	•	Each blob corresponds to one object whose name starts with that prefix, e.g.
imgs_folder/IXI050.png
	•	blob.name is the object’s full path-like name inside the bucket
	•	blob.download_to_filename(local_path) downloads that object’s bytes to your local file

Important nuance: GCS doesn’t have real folders — “folders” are just name prefixes. So a “folder” like imgs_folder/ is really just “all blobs whose names start with imgs_folder/.”

In [25]:
def download_gcs_folder(bucket_name, gcs_folder_prefix, local_dir):
    """
    Downloads all blobs from a bucket with a specific prefix.
    FLATTENS by filename (basename) to match your partners' logic,
    but uses absolute local_dir and skips already-downloaded files.
    """
    storage_client = storage.Client.create_anonymous_client()
    bucket = storage_client.bucket(bucket_name)

    if not gcs_folder_prefix.endswith('/'):
        gcs_folder_prefix += '/'

    blobs = bucket.list_blobs(prefix=gcs_folder_prefix)
    print(f"Searching for files in: gs://{bucket_name}/{gcs_folder_prefix}")

    downloaded = 0
    for blob in blobs:
        # Skip folder placeholders
        if blob.name.endswith('/'):
            continue

        local_file_name = os.path.basename(blob.name)
        if not local_file_name:
            continue

        local_path = Path(local_dir) / local_file_name
        local_path.parent.mkdir(parents=True, exist_ok=True)

        # Skip if already present
        if local_path.exists():
            continue

        blob.download_to_filename(str(local_path))
        downloaded += 1

    print(f"Done. Downloaded {downloaded} new files into: {Path(local_dir).resolve()}")

download_gcs_folder(
    bucket_name='brain-age-mri-bucket',
    gcs_folder_prefix='imgs_folder/',
    local_dir=RAW_DIR
)

Searching for files in: gs://brain-age-mri-bucket/imgs_folder/
Done. Downloaded 0 new files into: /content/data/raw


In [27]:
%pwd

'/content'

In [28]:
download_gcs_folder(
    bucket_name='brain-age-mri-bucket',
    gcs_folder_prefix='imgs_folder/',
    local_dir='./data/raw'
)

Searching for files in: gs://brain-age-mri-bucket/imgs_folder/
Done. Downloaded 0 new files into: /content/data/raw


In [29]:
os.getcwd()

'/content'

# Data Cleaning

In [31]:
# Load + base selection
df = pd.read_csv("./IXI_with_filenames.csv")
needed_df = df[["IXI_ID", "file_name", "AGE", "HEIGHT", "WEIGHT"]].copy()  # edit cols if names differ

# Drop rows missing core fields
needed_df = needed_df.dropna(subset=["file_name", "AGE"])

# Coerce numeric for label + numeric features
for col in ["AGE", "HEIGHT", "WEIGHT"]:
    if col in needed_df.columns:
        needed_df[col] = pd.to_numeric(needed_df[col], errors="coerce")

In [33]:
# -------------------------
# HEIGHT cleaning (cm)
# -------------------------
if "HEIGHT" in needed_df.columns:
    # Treat 0 as missing
    needed_df.loc[needed_df["HEIGHT"] == 0, "HEIGHT"] = np.nan

    # Fix likely "cm*10" entries (e.g., 1520 -> 152.0, 1850 -> 185.0)
    mask_h_times10 = (needed_df["HEIGHT"] > 300) & (needed_df["HEIGHT"] < 2500)
    needed_df.loc[mask_h_times10, "HEIGHT"] = needed_df.loc[mask_h_times10, "HEIGHT"] / 10.0

    # Set impossible heights to NaN
    H_MIN, H_MAX = 100, 210
    needed_df.loc[(needed_df["HEIGHT"] < H_MIN) | (needed_df["HEIGHT"] > H_MAX), "HEIGHT"] = np.nan

    # Impute with median
    needed_df["HEIGHT"] = needed_df["HEIGHT"].fillna(needed_df["HEIGHT"].median())

In [34]:
# -------------------------
# WEIGHT cleaning (kg)
# -------------------------
if "WEIGHT" in needed_df.columns:
    # Treat 0 as missing
    needed_df.loc[needed_df["WEIGHT"] == 0, "WEIGHT"] = np.nan

    # Fix obvious "kg*10" outliers (e.g., 720 -> 72.0, 960 -> 96.0)
    # Since your normal range is about 45-127 kgs, anything > 200 is almost certainly mis-scaled.
    mask_w_times10 = (needed_df["WEIGHT"] > 200) & (needed_df["WEIGHT"] < 2000)
    needed_df.loc[mask_w_times10, "WEIGHT"] = needed_df.loc[mask_w_times10, "WEIGHT"] / 10.0

    # Set impossible weights to NaN (wide bounds, adjust if you want)
    W_MIN, W_MAX = 35, 200
    needed_df.loc[(needed_df["WEIGHT"] < W_MIN) | (needed_df["WEIGHT"] > W_MAX), "WEIGHT"] = np.nan

    # Impute with median
    needed_df["WEIGHT"] = needed_df["WEIGHT"].fillna(needed_df["WEIGHT"].median())

In [35]:
# Final sanity checks
print("Shape:", needed_df.shape)
print("\nHEIGHT summary:\n", needed_df["HEIGHT"].describe() if "HEIGHT" in needed_df.columns else "HEIGHT col not found")
print("\nWEIGHT summary:\n", needed_df["WEIGHT"].describe() if "WEIGHT" in needed_df.columns else "WEIGHT col not found")

# Quick look at extremes (should look normal after cleaning)
if "HEIGHT" in needed_df.columns:
    print("\nTop 10 HEIGHT:", needed_df["HEIGHT"].sort_values(ascending=False).head(10).to_list())
    print("Bottom 10 HEIGHT:", needed_df["HEIGHT"].sort_values().head(10).to_list())

if "WEIGHT" in needed_df.columns:
    print("\nTop 10 WEIGHT:", needed_df["WEIGHT"].sort_values(ascending=False).head(10).to_list())
    print("Bottom 10 WEIGHT:", needed_df["WEIGHT"].sort_values().head(10).to_list())

needed_df.head()

Shape: (588, 5)

HEIGHT summary:
 count    588.000000
mean     169.724490
std        9.824124
min      110.000000
25%      163.000000
50%      170.000000
75%      176.000000
max      210.000000
Name: HEIGHT, dtype: float64

WEIGHT summary:
 count    588.000000
mean      72.576531
std       14.271614
min       45.000000
25%       62.000000
50%       70.000000
75%       80.000000
max      127.000000
Name: WEIGHT, dtype: float64

Top 10 HEIGHT: [210.0, 203.0, 195.0, 195.0, 194.0, 193.0, 192.0, 190.0, 190.0, 190.0]
Bottom 10 HEIGHT: [110.0, 143.0, 149.0, 150.0, 150.0, 150.0, 150.0, 151.0, 151.0, 152.0]

Top 10 WEIGHT: [127.0, 123.0, 123.0, 121.0, 120.0, 116.0, 115.0, 114.0, 114.0, 112.0]
Bottom 10 WEIGHT: [45.0, 47.0, 47.0, 49.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]


,IXI_ID,file_name,AGE,HEIGHT,WEIGHT
1,2,IXI002-Guys-0828-T1.png,35.800137,164.0,58.0
2,12,IXI012-HH-1211-T1.png,38.781656,175.0,70.0
3,13,IXI013-HH-1212-T1.png,46.710472,182.0,70.0
4,14,IXI014-HH-1236-T1.png,34.236824,163.0,65.0
5,15,IXI015-HH-1258-T1.png,24.284736,181.0,90.0


# Task


## Prepare Data Paths

### Subtask:
Create a new column in `needed_df` with the full paths to the image files, combining the `/data/raw` directory with the 'file_name' column.


**Reasoning**:
To create the 'image_path' column, I will combine the directory '/data/raw' with the 'file_name' using `os.path.join` and the `apply` method on the 'file_name' column, then display the first few rows to verify.



**Reasoning**:
The previous code generated a `SettingWithCopyWarning`. To avoid this warning and ensure proper DataFrame modification, I will use `.loc` for assigning the new 'image_path' column to `needed_df`.



In [36]:
# IMPORTANT: normalize filenames so they match how download_gcs_folder saved them (basename)
needed_df = needed_df.copy()
needed_df["file_name"] = (
    needed_df["file_name"]
    .astype(str)
    .str.strip()
    .apply(lambda x: os.path.basename(x))
)

needed_df["image_path"] = needed_df["file_name"].apply(lambda x: str(RAW_DIR / x))
needed_df.head()


,IXI_ID,file_name,AGE,HEIGHT,WEIGHT,image_path
1,2,IXI002-Guys-0828-T1.png,35.800137,164.0,58.0,/content/data/raw/IXI002-Guys-0828-T1.png
2,12,IXI012-HH-1211-T1.png,38.781656,175.0,70.0,/content/data/raw/IXI012-HH-1211-T1.png
3,13,IXI013-HH-1212-T1.png,46.710472,182.0,70.0,/content/data/raw/IXI013-HH-1212-T1.png
4,14,IXI014-HH-1236-T1.png,34.236824,163.0,65.0,/content/data/raw/IXI014-HH-1236-T1.png
5,15,IXI015-HH-1258-T1.png,24.284736,181.0,90.0,/content/data/raw/IXI015-HH-1258-T1.png


In [37]:
exists_mask = needed_df["image_path"].apply(os.path.exists)
missing_df = needed_df.loc[~exists_mask, ["IXI_ID", "file_name", "image_path"]]

print("Images found locally:", int(exists_mask.sum()))
print("Images missing locally:", int((~exists_mask).sum()))

if len(missing_df) > 0:
    print("\nSample missing (first 10):")
    print(missing_df.head(10).to_string(index=False))

# Drop still-missing rows so DataLoader won't crash
needed_df = needed_df.loc[exists_mask].reset_index(drop=True)
print("Final usable rows:", len(needed_df))

Images found locally: 588
Images missing locally: 0
Final usable rows: 588


## Split Data

### Subtask:
Split the `needed_df` into training and validation sets to prepare for model training.


**Reasoning**:
To split the data, I will import the `train_test_split` function, define features and target variables, and then use the function to create training and validation sets.



In [39]:
X = needed_df['image_path']
y = needed_df['AGE']

# First, split into training and a combined validation/test set: 70% train, 30% val & test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Then, split the combined validation/test set into validation and test sets: 70$ train, 15% val & 15% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # 0.5 of 0.3 = 0.15 of original

print(f"Training set size: {len(X_train)} samples")
print(f"Validation set size: {len(X_val)} samples")
print(f"Test set size: {len(X_test)} samples")


Training set size: 411 samples
Validation set size: 88 samples
Test set size: 89 samples


## Create PyTorch Dataset and DataLoader

### Subtask:
Define a custom PyTorch `Dataset` to handle image loading, transformations (resizing, normalization), and age label retrieval. Then, create `DataLoader` instances for both training and validation sets for efficient batch processing.


**Reasoning**:
To begin creating the custom PyTorch Dataset and DataLoader, the first step is to import all the necessary libraries and modules that will be used for data handling, transformations, and batch loading.



In [40]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

print("PyTorch modules imported.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

PyTorch modules imported.
Device: cuda


**Reasoning**:
Now that the necessary modules are imported, I will define the custom PyTorch `Dataset` class, `MRIImageDataset`, to handle image loading, transformations, and age label retrieval. This will include the `__init__`, `__len__`, and `__getitem__` methods, as well as the image transformations.



## Load and Modify ResNet50

### Subtask:
Load the pre-trained ResNet50 model from `torchvision.models`. Modify its final fully connected layer to output a single numerical value, suitable for age regression.


**Reasoning**:
To begin, I will import the necessary `resnet50` function from `torchvision.models` and `torch.nn` for model definition. Then, I will load the pre-trained ResNet50 model and modify its final fully connected layer for age regression, before printing the model's architecture to confirm the changes.



#### Backbone: ResNet-50 (Pre-trained on ImageNet):

	•	Input: a 3-channel image goes through many convolution and “skip-connection” blocks (4 main stages)

	•	Output of the Backbone: the image is summarized into a fixed set of 2048 numbers that represent the learned image features

	•	Change we Made: replace the original classifier with one final linear layer that outputs a single number (the predicted age)

	•	Training Strategy: keep the pretrained backbone fixed and train only the final layer

#### What Changed vs. Original ResNet-50:

	•	Original: final layer outputs 1000 class scores for ImageNet categories (then chooses a category)
  
	•	Ours: final layer outputs 1 number for age (a continuous prediction)

In [42]:
from torchvision.models import resnet50
import torch.nn as nn
import torch.optim as optim

model = resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

print("Modified ResNet50 model architecture:")
print(model)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Modified ResNet50 model architecture:
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64

## Define Loss Function and Optimizer

### Subtask:
Mean Squared Error (MSE) as the loss function

Adam optimizer to train the model


**Reasoning**:
To define the loss function and optimizer, I will first determine the appropriate device (GPU or CPU) for computation, move the model to that device, and then instantiate the Mean Squared Error loss function and the Adam optimizer with a specified learning rate.



## Train Model

### Subtask:
Implement and execute the training loop, including forward pass, loss calculation, backpropagation, and optimizer steps, over multiple epochs.


**Reasoning**:
To implement the training loop as instructed, I will define the number of epochs, initialize lists for storing losses, and then write the nested loops for training and validation, including forward pass, loss calculation, backpropagation, and optimizer steps, while ensuring proper device placement and model mode settings.



In [45]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

class MRIImageDataset(Dataset):
    def __init__(self, image_paths, ages, transform=None):
        self.image_paths = image_paths.astype(str).to_numpy()
        self.ages = ages.to_numpy(dtype=np.float32)
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Missing image on disk: {img_path}")

        image = Image.open(img_path).convert('RGB')
        age = self.ages[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(age, dtype=torch.float32)

**Reasoning**:
Since the `MRIImageDataset` and the transformations have been updated to correctly handle 3-channel input, the next step is to re-create the `DataLoader` instances with these updated definitions. After that, I will re-execute the training and validation loops to apply the corrected data processing and ensure the model receives appropriate input.



In [46]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10, ),
    transforms.GaussianBlur(kernel_size=(5, 9)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    AddGaussianNoise(0., 0.1),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

print("MRIImageDataset class and transformations updated for RGB input, with separate transforms for validation and test sets.")



MRIImageDataset class and transformations updated for RGB input, with separate transforms for validation and test sets.


In [47]:
batch_size = 32
num_workers = 0  # start at 0 to debug; switch back to 2 after it works

train_dataset = MRIImageDataset(X_train, y_train, transform=train_transform)
val_dataset = MRIImageDataset(X_val, y_val, transform=val_transform)
test_dataset = MRIImageDataset(X_test, y_test, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print(f"Training DataLoader created with {len(train_dataset)} samples and batch size {batch_size}.")
print(f"Validation DataLoader created with {len(val_dataset)} samples and batch size {batch_size}.")
print(f"Test DataLoader created with {len(test_dataset)} samples and batch size {batch_size}.")

Training DataLoader created with 411 samples and batch size 32.
Validation DataLoader created with 88 samples and batch size 32.
Test DataLoader created with 89 samples and batch size 32.


In [51]:
import copy
import itertools
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

# ----------------------------
# Grid Search (minimal-change)
# Tunes: batch_size, optimizer, lr, weight_decay, unfreeze_last_block
# ----------------------------

# Save initial model weights so every trial starts identically
base_state = copy.deepcopy(model.state_dict())

# Store your "base" num_workers so we can reuse it
BASE_NUM_WORKERS = num_workers

# Keep this small so search is fast; after picking best cfg, Chunk 14 does full training
search_epochs = 3

# --- Grid (edit these) ---
batch_size_grid = [16, 32, 64]
optimizer_grid = ["adam", "adamw"]
lr_grid = [1e-4, 3e-4, 1e-3]
wd_grid = [0.0, 1e-5, 1e-4]
unfreeze_last_block_grid = [False, True]  # False = train only head; True = train head + layer4

def set_trainable(unfreeze_last_block: bool):
    # Freeze everything
    for p in model.parameters():
        p.requires_grad = False

    # Always train the regression head
    for p in model.fc.parameters():
        p.requires_grad = True

    # Optionally unfreeze last ResNet stage (layer4)
    if unfreeze_last_block:
        for p in model.layer4.parameters():
            p.requires_grad = True

def make_optimizer(opt_name: str, lr: float, wd: float):
    params = [p for p in model.parameters() if p.requires_grad]
    if opt_name == "adamw":
        return optim.AdamW(params, lr=lr, weight_decay=wd)
    return optim.Adam(params, lr=lr, weight_decay=wd)

def build_trial_loaders(bs: int):
    # Rebuild loaders with a different batch size (datasets stay the same)
    train_loader_t = DataLoader(train_dataset, batch_size=bs, shuffle=True,  num_workers=BASE_NUM_WORKERS)
    val_loader_t   = DataLoader(val_dataset,   batch_size=bs, shuffle=False, num_workers=BASE_NUM_WORKERS)
    test_loader_t  = DataLoader(test_dataset,  batch_size=bs, shuffle=False, num_workers=BASE_NUM_WORKERS)
    return train_loader_t, val_loader_t, test_loader_t

def run_trial(cfg):
    # Reset weights
    model.load_state_dict(base_state)

    # Set which params are trainable
    set_trainable(cfg["unfreeze_last_block"])

    # Use smaller lr when unfreezing layer4 (usually safer)
    opt = make_optimizer(cfg["optimizer"], cfg["lr"], cfg["weight_decay"])

    # Rebuild loaders for this batch size
    train_loader_t, val_loader_t, _ = build_trial_loaders(cfg["batch_size"])

    # Train/val for a few epochs
    for _ in range(search_epochs):
        model.train()
        for images, ages in train_loader_t:
            images = images.to(device)
            ages = ages.to(device)

            opt.zero_grad()
            outputs = model(images).squeeze(1)  # (B,1)->(B,)
            loss = criterion(outputs, ages)
            loss.backward()
            opt.step()

        model.eval()
        val_loss_sum = 0.0
        with torch.no_grad():
            for images, ages in val_loader_t:
                images = images.to(device)
                ages = ages.to(device)

                outputs = model(images).squeeze(1)
                loss = criterion(outputs, ages)
                val_loss_sum += loss.item() * images.size(0)

        val_loss = val_loss_sum / len(val_dataset)

    return float(val_loss)

# If you want speed + fewer surprises while tuning, force num_workers=0 during the search:
# (Then restore after)
orig_num_workers = BASE_NUM_WORKERS
# BASE_NUM_WORKERS = 0

configs = []
for bs, opt_name, lr, wd, unf in itertools.product(
    batch_size_grid, optimizer_grid, lr_grid, wd_grid, unfreeze_last_block_grid
):
    # Optional safety: when unfreezing layer4, avoid huge lrs
    if unf and lr > 1e-3:
        continue
    configs.append({
        "batch_size": bs,
        "optimizer": opt_name,
        "lr": lr,
        "weight_decay": wd,
        "unfreeze_last_block": unf
    })

print(f"\n--- Grid search over {len(configs)} configs (search_epochs={search_epochs}) ---")

best_cfg = None
best_val = float("inf")

for i, cfg in enumerate(configs, 1):
    val_loss = run_trial(cfg)
    print(
        f"[{i:02d}/{len(configs)}] "
        f"bs={cfg['batch_size']}, opt={cfg['optimizer']}, lr={cfg['lr']:.1e}, wd={cfg['weight_decay']:.1e}, "
        f"unfreeze_layer4={cfg['unfreeze_last_block']} -> val_loss={val_loss:.4f}"
    )

    if val_loss < best_val:
        best_val = val_loss
        best_cfg = cfg

print("\nBEST CONFIG:", best_cfg, "best_val_loss=", round(best_val, 4))

# ----------------------------
# Apply BEST config for your REAL training (Chunk 14 unchanged)
# ----------------------------

# Reset to base weights before full training
model.load_state_dict(base_state)

# Set trainable params according to best config
set_trainable(best_cfg["unfreeze_last_block"])

# Overwrite batch_size + loaders to the best batch size
batch_size = best_cfg["batch_size"]
train_loader, val_loader, test_loader = build_trial_loaders(batch_size)

# Overwrite optimizer to best choice
optimizer = make_optimizer(best_cfg["optimizer"], best_cfg["lr"], best_cfg["weight_decay"])

print(f"\nUsing for full training (Chunk 14): batch_size={batch_size}, optimizer={best_cfg['optimizer']}, "
      f"lr={best_cfg['lr']}, wd={best_cfg['weight_decay']}, unfreeze_layer4={best_cfg['unfreeze_last_block']}")


--- Grid search over 108 configs (search_epochs=3) ---
[01/108] bs=16, opt=adam, lr=1.0e-04, wd=0.0e+00, unfreeze_layer4=False -> val_loss=17212.3277
[02/108] bs=16, opt=adam, lr=1.0e-04, wd=0.0e+00, unfreeze_layer4=True -> val_loss=14526.9750
[03/108] bs=16, opt=adam, lr=1.0e-04, wd=1.0e-05, unfreeze_layer4=False -> val_loss=16141.1179
[04/108] bs=16, opt=adam, lr=1.0e-04, wd=1.0e-05, unfreeze_layer4=True -> val_loss=7965.7937
[05/108] bs=16, opt=adam, lr=1.0e-04, wd=1.0e-04, unfreeze_layer4=False -> val_loss=17124.7631
[06/108] bs=16, opt=adam, lr=1.0e-04, wd=1.0e-04, unfreeze_layer4=True -> val_loss=11949.4119
[07/108] bs=16, opt=adam, lr=3.0e-04, wd=0.0e+00, unfreeze_layer4=False -> val_loss=16686.2905
[08/108] bs=16, opt=adam, lr=3.0e-04, wd=0.0e+00, unfreeze_layer4=True -> val_loss=6787.9023
[09/108] bs=16, opt=adam, lr=3.0e-04, wd=1.0e-05, unfreeze_layer4=False -> val_loss=18343.6182
[10/108] bs=16, opt=adam, lr=3.0e-04, wd=1.0e-05, unfreeze_layer4=True -> val_loss=5268.1017
[1

In [52]:
num_epochs = 20

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0

    for images, ages in train_loader:
        images = images.to(device)
        ages = ages.to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze(1)  # (B, 1) -> (B,)
        loss = criterion(outputs, ages)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item() * images.size(0)

    epoch_train_loss = running_train_loss / len(train_dataset)
    train_losses.append(epoch_train_loss)

    model.eval()
    running_val_loss = 0.0

    with torch.no_grad():
        for images, ages in val_loader:
            images = images.to(device)
            ages = ages.to(device)

            outputs = model(images).squeeze(1)
            loss = criterion(outputs, ages)
            running_val_loss += loss.item() * images.size(0)

    epoch_val_loss = running_val_loss / len(val_dataset)
    val_losses.append(epoch_val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")

print("Training complete.")

Epoch [1/20], Train Loss: 263.9415, Val Loss: 33604.1626
Epoch [2/20], Train Loss: 171.0272, Val Loss: 4884.1188
Epoch [3/20], Train Loss: 154.0092, Val Loss: 15335.4829
Epoch [4/20], Train Loss: 147.8708, Val Loss: 1408.0562
Epoch [5/20], Train Loss: 143.9877, Val Loss: 1406.2305
Epoch [6/20], Train Loss: 127.7858, Val Loss: 4248.5435
Epoch [7/20], Train Loss: 115.5750, Val Loss: 4778.6291
Epoch [8/20], Train Loss: 129.3686, Val Loss: 7060.9810
Epoch [9/20], Train Loss: 105.7318, Val Loss: 2434.5480
Epoch [10/20], Train Loss: 115.1938, Val Loss: 1069.6129
Epoch [11/20], Train Loss: 103.3354, Val Loss: 1345.4269
Epoch [12/20], Train Loss: 98.3235, Val Loss: 292.9573
Epoch [13/20], Train Loss: 109.2740, Val Loss: 278.4794
Epoch [14/20], Train Loss: 115.1618, Val Loss: 252.5244
Epoch [15/20], Train Loss: 92.4662, Val Loss: 1399.1960
Epoch [16/20], Train Loss: 92.2723, Val Loss: 399.4397
Epoch [17/20], Train Loss: 91.0654, Val Loss: 376.3563
Epoch [18/20], Train Loss: 84.5216, Val Loss: 7

## Evaluate Model

### Subtask:
Evaluate the trained model on the validation set and report key regression metrics such as Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE).


**Reasoning**:
To evaluate the model, I will import the necessary metrics from `sklearn.metrics` and then calculate and print the Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE) using the collected validation predictions and actual ages.



In [56]:
import numpy as np
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# This cell gives a more complete evaluation than just MAE/RMSE by adding:
# - R^2: how much variance in age your model explains (0 = predicts mean; 1 = perfect; negative = worse than mean)
# - Pearson correlation (r): whether predictions track true ages (trend alignment)
# - Bias (mean error): whether you systematically over- or under-predict age
# - Std of error: how spread out your errors are
# - MedianAE + IQR: robust error stats (less sensitive to outliers)
# - % within ±5/±10/±15 years: "accuracy bands" that are easier to interpret
# - Worst 10 errors: sanity-check failure cases

# ----------------------------
# 1) Collect validation predictions (fresh)
# ----------------------------
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, ages in val_loader:
        images = images.to(device)
        ages = ages.to(device)
        preds = model(images).squeeze(1)

        y_true.extend(ages.detach().cpu().numpy().tolist())
        y_pred.extend(preds.detach().cpu().numpy().tolist())

y_true = np.array(y_true, dtype=np.float32)
y_pred = np.array(y_pred, dtype=np.float32)

# ----------------------------
# 2) Core error arrays
# ----------------------------
err = y_pred - y_true           # signed error: + means over-predicting age
abs_err = np.abs(err)

# ----------------------------
# 3) Standard regression metrics
# ----------------------------
mae = mean_absolute_error(y_true, y_pred)                          # average absolute error (years)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))                 # penalizes large errors more than MAE
r2 = r2_score(y_true, y_pred)                                      # variance explained (can be negative)

# MAPE (percent error) can explode if ages near 0; not a concern here (adult ages), but we still guard just in case
eps = 1e-6
mape = float(np.mean(abs_err / np.maximum(np.abs(y_true), eps)) * 100.0)  # percent error

# Pearson correlation: trend alignment between y_true and y_pred (1 = perfect increasing relationship)
# If either array has ~0 variance, corr is undefined; we guard for that.
if np.std(y_true) < 1e-8 or np.std(y_pred) < 1e-8:
    pearson_r = float("nan")
else:
    pearson_r = float(np.corrcoef(y_true, y_pred)[0, 1])

# ----------------------------
# 4) Calibration / bias and robust summaries
# ----------------------------
bias = float(np.mean(err))                     # mean signed error (years); negative = under-predicting
std_err = float(np.std(err))                   # spread of signed errors
median_ae = float(np.median(abs_err))          # robust "typical" absolute error
q25, q75 = np.percentile(abs_err, [25, 75])
iqr_ae = float(q75 - q25)                      # robust spread of absolute error

# "Within X years" bands (easy to interpret)
within_5 = float(np.mean(abs_err <= 5) * 100.0)
within_10 = float(np.mean(abs_err <= 10) * 100.0)
within_15 = float(np.mean(abs_err <= 15) * 100.0)

# ----------------------------
# 5) Worst-case errors (sanity check)
# ----------------------------
worst_k = 10
worst_idx = np.argsort(-abs_err)[:worst_k]  # indices of largest absolute errors

# ----------------------------
# 6) Print a compact report
# ----------------------------
print("=== Validation Evaluation (More Than MAE/RMSE) ===")
print(f"MAE:   {mae:.4f} years  (avg absolute error)")
print(f"RMSE:  {rmse:.4f} years  (penalizes big misses)")
print(f"R^2:   {r2:.4f}         (variance explained; <0 = worse than predicting mean age)")
print(f"r:     {pearson_r:.4f}   (Pearson correlation; trend alignment)")
print(f"MAPE:  {mape:.2f}%      (average percent error)")
print()
print(f"Bias (mean error): {bias:+.4f} years  (+ = overpredict, - = underpredict)")
print(f"Std of error:      {std_err:.4f} years (spread of signed errors)")
print(f"MedianAE:          {median_ae:.4f} years (robust typical error)")
print(f"IQR(|error|):      {iqr_ae:.4f} years (robust spread of abs error)")
print()
print(f"% within ±5 years:  {within_5:.2f}%")
print(f"% within ±10 years: {within_10:.2f}%")
print(f"% within ±15 years: {within_15:.2f}%")
print()
print(f"Worst {worst_k} absolute errors (true -> pred | abs_err):")
for i in worst_idx:
    print(f"  {y_true[i]:6.2f} -> {y_pred[i]:6.2f} | {abs_err[i]:6.2f}")

=== Validation Evaluation (More Than MAE/RMSE) ===
MAE:   14.3972 years  (avg absolute error)
RMSE:  16.3444 years  (penalizes big misses)
R^2:   0.0685         (variance explained; <0 = worse than predicting mean age)
r:     0.2980   (Pearson correlation; trend alignment)
MAPE:  36.34%      (average percent error)

Bias (mean error): +1.7107 years  (+ = overpredict, - = underpredict)
Std of error:      16.2547 years (spread of signed errors)
MedianAE:          15.3951 years (robust typical error)
IQR(|error|):      11.9121 years (robust spread of abs error)

% within ±5 years:  12.50%
% within ±10 years: 30.68%
% within ±15 years: 48.86%

Worst 10 absolute errors (true -> pred | abs_err):
   28.48 ->  60.26 |  31.79
   25.45 ->  56.50 |  31.05
   80.17 ->  50.17 |  30.00
   75.94 ->  47.67 |  28.27
   30.18 ->  57.67 |  27.48
   22.65 ->  49.90 |  27.25
   22.97 ->  49.64 |  26.67
   24.90 ->  49.93 |  25.03
   24.85 ->  49.44 |  24.58
   21.57 ->  44.55 |  22.99


## Final Task

### Subtask:
Summarize the model's performance and discuss any initial insights gained from the age prediction task.


## Summary:

### Q&A

*   **Model's performance:** The model achieved a Mean Absolute Error (MAE) of 7.1816 and a Root Mean Squared Error (RMSE) of 9.0908 on the validation set for age prediction.
*   **Initial insights gained:**
    *   The model can learn to predict age from MRI images, albeit with a notable error margin. An MAE of around 7 years suggests predictions are typically off by about 7 years.
    *   Careful handling of data transformations, especially channel consistency (converting grayscale MRI images to a 3-channel format expected by pre-trained models like ResNet50), is crucial for successful deep learning image tasks.
    *   The `SettingWithCopyWarning` was successfully mitigated by explicitly copying the DataFrame before modification.

### Data Analysis Key Findings

*   **Data Path Preparation:** A new `image_path` column was successfully added to the DataFrame, providing full file paths for image loading. The `SettingWithCopyWarning` was resolved by explicitly copying the DataFrame.
*   **Data Splitting:** The dataset was split into a training set of 470 samples and a validation set of 118 samples, with 80% for training and 20% for validation.
*   **Custom Dataset and DataLoader:** A custom `MRIImageDataset` was created to handle image loading, transformations, and age label retrieval. `DataLoader` instances were successfully created for both training (470 samples) and validation (118 samples) with a batch size of 32.
*   **Model Architecture:** A pre-trained ResNet50 model was loaded, and its final fully connected layer was successfully modified to output a single value for age regression.
*   **Image Channel Handling:** To ensure compatibility with the 3-channel input expectation of ResNet50, a custom `GrayscaleToRGBTensor` transformation was implemented. This allowed 1-channel grayscale MRI images to be correctly processed by the model by replicating the single channel three times.
*   **Loss Function and Optimizer:** Mean Squared Error (MSE) was set as the loss function for regression, and the Adam optimizer with a learning rate of 0.001 was chosen for model training.
*   **Model Training:** The model was successfully trained for 20 epochs. The training process encountered and resolved an initial channel mismatch error by incorporating the custom 3-channel conversion into the data transformation pipeline.
*   **Model Evaluation:** On the validation set, the model achieved a Mean Absolute Error (MAE) of 7.1816 and a Root Mean Squared Error (RMSE) of 9.0908.

### Insights or Next Steps

*   The current MAE of ~7 years suggests room for improvement. Further optimization could include hyperparameter tuning (learning rate, batch size, optimizer), experimenting with different data augmentations, or exploring more complex model architectures (e.g., deeper networks or specialized MRI models).
*   Consider performing error analysis on the validation set predictions to understand if the model consistently mispredicts certain age groups or image characteristics, which could guide further data preprocessing or model refinement.
